In [1]:
import os, csv
from dotenv import load_dotenv
import pandas as pd
import time

from datetime import datetime
from xAPIConnector import *
import schedule
import requests

import calendar
import pandas as pd
import pandas_ta as ta
import numpy as np

from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.foreignexchange import ForeignExchange
load_dotenv()

True

In [2]:
XTB_USER_ID = os.getenv('XTB_USER_ID')
XTB_USER_PASSWORD = os.getenv('XTB_USER_PASSWORD')

API_KEYS = os.getenv('ALPHA_KEYS')
cc = ForeignExchange(key=API_KEYS)

LINE_TOKEN = os.getenv('LINE_TOKEN')
LINE_URL = 'https://notify-api.line.me/api/notify'
LINE_HEADERS = {'content-type':'application/x-www-form-urlencoded','Authorization':'Bearer ' + LINE_TOKEN }

ts = TimeSeries(key=API_KEYS, output_format='pandas')
# XTB_USER_ID, XTB_USER_PASSWORD
# client = APIClient()

In [3]:
client = APIClient(port=5112)

loginResponse = client.execute(loginCommand(userId=XTB_USER_ID, password=XTB_USER_PASSWORD))
logger.info(str(loginResponse)) 

# check if user logged in correctly
if(loginResponse['status'] == False):
    print('Login failed. Error code: {0}'.format(loginResponse['errorCode']))
    
# get ssId from login response
ssid = loginResponse['streamSessionId']

[2022-05-11 12:51:23,666][connect:74] Socket connected
[2022-05-11 12:51:23,668][_waitingSend:88] Sent: b'{"command": "login", "arguments": {"userId": "1959587", "password": "veGan/1102199c", "appName": ""}}'
[2022-05-11 12:51:24,101][_read:107] Received: {'status': True, 'streamSessionId': 'ca005bfffe1f3af5-000088aa-002d6d07-bc7a222d149ba105-36098bfd'}
[2022-05-11 12:51:24,102][<module>:4] {'status': True, 'streamSessionId': 'ca005bfffe1f3af5-000088aa-002d6d07-bc7a222d149ba105-36098bfd'}


In [4]:
ping_resp = client.execute(pingCommand())
ping_resp['status']

[2022-05-10 11:15:22,936][_waitingSend:88] Sent: b'{"command": "ping", "arguments": {}}'
[2022-05-10 11:15:23,152][_read:107] Received: {'status': True}


True

In [7]:
client.execute(getTickPricesCommand('EURUSD', calendar.timegm(time.gmtime()) * 100))

[2022-05-10 08:59:54,208][_waitingSend:88] Sent: b'{"command": "getTickPrices", "arguments": {"level": 0, "symbols": ["EURUSD"], "timestamp": 165214799400}}'
[2022-05-10 08:59:54,421][_read:107] Received: {'status': True, 'returnData': {'quotations': [{'symbol': 'EURUSD', 'ask': 1.05626, 'bid': 1.05612, 'high': 1.05685, 'low': 1.0552, 'askVolume': 50000, 'bidVolume': 200000, 'timestamp': 1652147994910, 'level': 0, 'exemode': 1, 'spreadRaw': 0.00014, 'spreadTable': 1.4}]}}


{'status': True,
 'returnData': {'quotations': [{'symbol': 'EURUSD',
    'ask': 1.05626,
    'bid': 1.05612,
    'high': 1.05685,
    'low': 1.0552,
    'askVolume': 50000,
    'bidVolume': 200000,
    'timestamp': 1652147994910,
    'level': 0,
    'exemode': 1,
    'spreadRaw': 0.00014,
    'spreadTable': 1.4}]}}

In [ ]:
order_resp = client.execute(newPendingOrderCommand(
    symbol='GOLD', 
    action='buy stop', 
    price=2000, 
    tp=2006, 
    sl=1995, 
    volume=0.01
))
order_resp

In [4]:
order_resp = client.execute(stopOrderCommand(
    orderId=369035605,
    symbol='GOLD', 
    action='buy', 
    price=2000,
    volume=0.01
))
order_resp

[2022-03-28 13:46:42,458][_waitingSend:88] Sent: b'{"command": "tradeTransaction", "arguments": {"tradeTransInfo": {"order": 369035605, "symbol": "GOLD"}}}'
[2022-03-28 13:46:42,699][_read:107] Received: {'status': False, 'errorCode': 'BE4002', 'errorDescr': 'Invalid nominal'}


{'status': False, 'errorCode': 'BE4002', 'errorDescr': 'Invalid nominal'}

In [1]:
client.disconnect()

NameError: name 'client' is not defined

In [3]:
class RsiO2Notify:
    def __init__(self, params, ts): #, client):
        self.ts = ts
        self.client = None # = client
        self.pair = params['pair']
        self.tf = params['tf']
        
        self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                      'close': None, 'T/P': None, 'S/L': None,'result': None}
        self.current_action = 'close'
        self.atr_len = params['atr_len']
        self.rsi_len = params['rsi_len']
        
        self.ema_len = params['ema_len']
        self.overbought = params['overbought']
        self.oversold = params['oversold']
        self.pip = params['pip']
        
        self.pip_value = params['pip_value']
        self.rr = params['rr']
        
        self.lot_size = 0.01
        self.order_id = 0
        self.target = 10
        self.round = params['round']
        
        self.line_url = params['line_url']
        self.line_token = params['line_token']
        self.line_headers = {'content-type':'application/x-www-form-urlencoded','Authorization':'Bearer ' + self.line_token }
        self.df = None
        
    def notifyMsg(self, *msg):
        r = requests.post(self.line_url, headers=self.line_headers, data={'message': msg[0]})
        print(r.text)
        
    def calLotSize(self, entry, exit):
        self.lot_size = abs(round(self.target / (((entry - exit) / self.pip ) * self.pip_value ) , 2))
        print(f'lot_size: {str(self.lot_size)}')
        
    def executeOrder(self, action, entry=0, exit=0, sl=0, tp=0):
        self.client = APIClient(port=5112)

        loginResponse = self.client.execute(loginCommand(userId=XTB_USER_ID, password=XTB_USER_PASSWORD))
        logger.info(str(loginResponse)) 

        # check if user logged in correctly
        if(loginResponse['status'] == False):
            print('Login failed. Error code: {0}'.format(loginResponse['errorCode']))
        
        now = calendar.timegm(time.gmtime()) * 100
        if action == 'buy':
            order_action='buy' # 'buy stop'
            tick_resp = self.client.execute(getTickPricesCommand(self.pair, now))
#             if tick_resp['returnData']['quotations'][0]['ask'] > round(entry, self.round):
#                 order_action='buy limit'
            print(f"{order_action} price: {str(round(entry, self.round))}, \
                  ask: {str(tick_resp['returnData']['quotations'][0]['ask'])}, \
                  bid: {str(tick_resp['returnData']['quotations'][0]['bid'])}")
                
            sl = tick_resp['returnData']['quotations'][0]['ask'] - (entry - sl)
            tp = tick_resp['returnData']['quotations'][0]['ask'] + (tp - entry)
                
            order_resp = self.client.execute(newPendingOrderCommand(
                symbol=self.pair, 
                action=order_action, 
                price=tick_resp['returnData']['quotations'][0]['ask'] , # round(entry, self.round), 
                tp=round(tp, self.round), 
                sl=round(sl, self.round), 
                volume=self.lot_size
            ))
            self.order_id = order_resp['returnData']['order']
            
        elif action == 'sell':
            order_action='sell' # 'sell stop'
            tick_resp = self.client.execute(getTickPricesCommand(self.pair, now))
#             if tick_resp['returnData']['quotations'][0]['bid'] < round(entry, self.round):
#                 order_action='sell limit'
                
            print(f"{order_action} price: {str(round(entry, self.round))}, \
                  ask: {str(tick_resp['returnData']['quotations'][0]['ask'])}, \
                  bid: {str(tick_resp['returnData']['quotations'][0]['bid'])}")
                
            sl = tick_resp['returnData']['quotations'][0]['bid'] + (sl - entry)
            tp = tick_resp['returnData']['quotations'][0]['bid'] - (entry - tp)
                
            order_resp = self.client.execute(newPendingOrderCommand(
                symbol=self.pair, 
                action= order_action, 
                price=tick_resp['returnData']['quotations'][0]['bid'], # round(entry, self.round), 
                tp=round(tp, self.round), 
                sl=round(sl, self.round), 
                volume=self.lot_size
            ))
            self.order_id = order_resp['returnData']['order']
            
        elif action == 'close buy':
            order_resp = client.execute(stopOrderCommand(
                orderId=self.order_id,
                symbol=self.pair, 
                action='buy', 
                price=round(exit, self.round),
                volume=self.lot_size
            ))
            
        elif action == 'close sell':
            order_resp = client.execute(stopOrderCommand(
                orderId=self.order_id,
                symbol=self.pair, 
                action='sell', 
                price=round(exit, self.round),
                volume=self.lot_size
            ))
            
        self.client.disconnect()
        
    def initial_bars(self):
        bars, _ = self.ts.get_intraday(symbol=self.pair,interval=self.tf, outputsize='full')
        df = pd.DataFrame({
            'timestamp': bars.index,
            'open': bars['1. open'],
            'high': bars['2. high'],
            'low': bars['3. low'],
            'close': bars['4. close'],
            'volumn': bars['5. volume']
        }) # convert alpha vantage data to use with strategy

        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df = df.iloc[::-1] # reverse row
        df.reset_index(drop=True, inplace=True)

        self.df = df
    
    def get_realtime(self):
        bars, _ = self.ts.get_intraday(symbol=self.pair,interval=self.tf, outputsize='compact')
        incoming_df = pd.DataFrame({
            'timestamp': bars.index,
            'open': bars['1. open'],
            'high': bars['2. high'],
            'low': bars['3. low'],
            'close': bars['4. close'],
            'volumn': bars['5. volume']
        }) # convert alpha vantage data to use with rsi strategy

        incoming_df['timestamp'] = pd.to_datetime(incoming_df['timestamp'], unit='ms')
     
        if (incoming_df['timestamp'].iloc[0] != self.df['timestamp'].iloc[-1]):
            self.df = self.df[:900].append(incoming_df.head(1), ignore_index = True)

        rsi_o2 = ta.Strategy(
            name = 'RSI 14 Days for opposite trade',
            description = 'RSI for alert',
            ta = [
                {'kind': 'ema', 'length': self.ema_len},
                {'kind': 'rsi', 'length': self.rsi_len},
                {'kind': 'atr', 'length': self.atr_len}
            ]
        )
        self.df.ta.strategy(rsi_o2)
    
        row = self.df.tail(1).to_dict('records')[0]
        
        if self.current_action == 'close' or self.order == {}:
            if (row['EMA_'+str(self.ema_len)] < row['low']) and (row['RSI_'+str(self.rsi_len)] > self.overbought):
#                 row['action'] = 'buy'
                self.order['action'] = 'buy'
                self.order['open time'] = row['timestamp']
                self.order['open'] = row['open']
                
                self.order['S/L'] = row['low']- row['ATRr_'+str(self.atr_len)]
                self.order['T/P'] = row['open'] + (self.rr * abs(row['open'] - self.order['S/L']))
                self.current_action = 'buy'
                self.notifyMsg(f"Buy {self.pair} RSI_O2\nprice: {str(self.order['open'])}\nTP: {str(self.order['T/P'])}\nSL: {str(self.order['S/L'])}")
                
                self.calLotSize(self.order['open'], self.order['T/P'])
                self.executeOrder('buy', entry=self.order['open'], exit=self.order['T/P'], sl=self.order['S/L'], tp=self.order['T/P'])
#                 print(f"Buy {self.pair} \nopen: {str(self.order['open'])}\nclose: {str(self.order['close'])}\n " + \
#                       f"ema: {str(row['EMA_'+str(self.ema_len)])}, rsi: {str(row['RSI_'+str(self.rsi_len)] )}")
                
            elif (row['EMA_'+str(self.ema_len)] > row['high']) and (row['RSI_'+str(self.rsi_len)] < self.oversold):   
#                 row['action'] = 'sell'
                self.order['action'] = 'sell'
                self.order['open time'] = row['timestamp']
                self.order['open'] = row['open']
                
                self.order['S/L'] = row['high'] + row['ATRr_'+str(self.atr_len)]
                self.order['T/P'] = row['open'] - (self.rr * abs(row['open'] - self.order['S/L']))
                self.current_action = 'sell'
                self.notifyMsg(f"Sell {self.pair} RSI_O2\nprice: {str(self.order['open'])}\nTP: {str(self.order['T/P'])}\nSL: {str(self.order['S/L'])}")
                
                self.calLotSize(self.order['open'], self.order['T/P'])
                self.executeOrder('sell', entry=self.order['open'], exit=self.order['T/P'], sl=self.order['S/L'], tp=self.order['T/P'])
#                 print(f"Sell {self.pair} \nopen: {str(self.order['open'])}\nclose: {str(self.order['close'])}\n " + \
#                       f"ema: {str(row['EMA_'+str(self.ema_len)])}, rsi: {str(row['RSI_'+str(self.rsi_len)] )}")

        if (self.current_action == 'buy'):
            if (row['low'] <= self.order['S/L']):
                self.order['result'] = 'S/L'
                self.order['close time'] = row['timestamp']
                self.order['close'] = self.order['S/L']
                
                self.current_action = 'close'
                self.notifyMsg(f"S/L Buy {self.pair} RSI_O2\nprice: {self.order['close']}")
                self.executeOrder('sell', entry=self.order['open'], exit=self.order['T/P'], sl=self.order['S/L'], tp=self.order['T/P'])
                self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                              'close': None, 'T/P': None, 'S/L': None,'result': None}
                
            elif (row['high'] >= self.order['T/P']):
                self.order['result'] = 'T/P'
                self.order['close time'] = row['timestamp']
                self.order['close'] = self.order['T/P']
                
                self.current_action = 'close'
                self.notifyMsg(f"T/P Buy {self.pair} RSI_O2\nprice: {self.order['close']}")
                self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                              'close': None, 'T/P': None, 'S/L': None,'result': None}

        elif (self.current_action == 'sell'):
            if (row['high'] >= self.order['S/L']):
                self.order['result'] = 'S/L'
                self.order['close time'] = row['timestamp']
                self.order['close'] = self.order['S/L']
                
                self.current_action = 'close'
                self.notifyMsg(f"S/L Sell {self.pair} RSI_O2\nprice: {self.order['close']}")
                self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                              'close': None, 'T/P': None, 'S/L': None,'result': None}
                
            elif (row['low'] <= self.order['T/P']):
                self.order['result'] = 'T/P'
                self.order['close time'] = row['timestamp']
                self.order['close'] = self.order['T/P']
                
                self.current_action = 'close'
                self.notifyMsg(f"T/P Sell {self.pair} RSI_O2\nprice: {self.order['close']}")
                self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                              'close': None, 'T/P': None, 'S/L': None,'result': None}

In [4]:
def keepConnectionAlive(client_obj):
    try:
        ping_resp = client_obj.execute(pingCommand())
        print(f"Connection Status: {ping_resp['status']}")
        if ping_resp['status'] == False:
            raise('Connection Status: False')
    except:
        r = requests.post(LINE_URL, headers=LINE_HEADERS, data={'message': 'Xtb connection lost!'})
        print(r.text)
        
#         loginResponse = client_obj.execute(loginCommand(userId=XTB_USER_ID, password=XTB_USER_PASSWORD))
#         logger.info(str(loginResponse)) 

#         # check if user logged in correctly
#         if(loginResponse['status'] == False):
#             print('Login failed. Error code: {0}'.format(loginResponse['errorCode']))

In [5]:
eurusd = RsiO2Notify({
    'pair': 'EURUSD',
    'tf': '15min',
    'atr_len': 19,
    'rsi_len': 10,
    'ema_len': 200,
    'overbought': 80,
    'oversold': 20,
    'pip': 0.0001,
    'pip_value': 10,
    'rr': 1.5,
    'round': 5,
    'line_url': LINE_URL,
    'line_token': LINE_TOKEN,
}, ts) #, client)

audusd = RsiO2Notify({
    'pair': 'AUDUSD',
    'tf': '15min',
    'atr_len': 16,
    'rsi_len': 11,
    'ema_len': 200,
    'overbought': 78,
    'oversold': 22,
    'pip': 0.0001,
    'pip_value': 10,
    'rr': 1.5,
    'round': 5,
    'line_url': LINE_URL,
    'line_token': LINE_TOKEN,
}, ts) #, client)

gbpusd = RsiO2Notify({
    'pair': 'GBPUSD',
    'tf': '15min',
    'atr_len': 10,
    'rsi_len': 20,
    'ema_len': 200,
    'overbought': 70,
    'oversold': 30,
    'pip': 0.0001,
    'pip_value': 10,
    'rr': 1.5,
    'round': 5,
    'line_url': LINE_URL,
    'line_token': LINE_TOKEN,
}, ts) #, client)

usdjpy = RsiO2Notify({
    'pair': 'USDJPY',
    'tf': '15min',
    'atr_len': 17,
    'rsi_len': 12,
    'ema_len': 200,
    'overbought': 73,
    'oversold': 27,
    'pip': 0.01,
    'pip_value': 8.04,
    'rr': 1.5,
    'round': 3,
    'line_url': LINE_URL,
    'line_token': LINE_TOKEN,
}, ts) #, client)

In [6]:
eurusd.initial_bars()
audusd.initial_bars()
# usdjpy.initial_bars()
# gbpusd.initial_bars()

In [7]:
def get_all_realtime():
#     keepConnectionAlive(client)
    eurusd.get_realtime()
    audusd.get_realtime()
#     usdjpy.get_realtime()
    
schedule.every(60 * 15).seconds.do(get_all_realtime) 

Every 900 seconds do get_all_realtime() (last run: [never], next run: 2022-05-12 07:50:32)

In [ ]:
while True:
    schedule.run_pending()
    time.sleep(1)

{"status":200,"message":"ok"}
lot_size: 0.06


[2022-05-12 12:53:14,841][connect:74] Socket connected
[2022-05-12 12:53:14,845][_waitingSend:88] Sent: b'{"command": "login", "arguments": {"userId": "1959587", "password": "veGan/1102199c", "appName": ""}}'
[2022-05-12 12:53:15,648][_read:107] Received: {'status': True, 'streamSessionId': '262540fffee58c68-00004899-004b44d0-66414dad4f566728-870ae3d2'}
[2022-05-12 12:53:15,649][executeOrder:44] {'status': True, 'streamSessionId': '262540fffee58c68-00004899-004b44d0-66414dad4f566728-870ae3d2'}
[2022-05-12 12:53:15,651][_waitingSend:88] Sent: b'{"command": "getTickPrices", "arguments": {"level": 0, "symbols": ["EURUSD"], "timestamp": 165233479500}}'
[2022-05-12 12:53:15,882][_read:107] Received: {'status': True, 'returnData': {'quotations': [{'symbol': 'EURUSD', 'ask': 1.05057, 'bid': 1.05043, 'high': 1.05287, 'low': 1.05016, 'askVolume': 300000, 'bidVolume': 200000, 'timestamp': 1652334795296, 'level': 0, 'exemode': 1, 'spreadRaw': 0.00014, 'spreadTable': 1.4}]}}
[2022-05-12 12:53:15,8

sell price: 1.0509,                   ask: 1.05057,                   bid: 1.05043


[2022-05-12 12:53:16,115][_read:107] Received: {'status': True, 'returnData': {'order': 624192805}}
[2022-05-12 12:53:16,116][close:115] Closing socket


{"status":200,"message":"ok"}
lot_size: 0.04


...

[2022-05-12 15:09:28,327][close:115] Closing socket


{"status":200,"message":"ok"}
{"status":200,"message":"ok"}
lot_size: 0.04


[2022-05-12 16:25:01,246][connect:74] Socket connected
[2022-05-12 16:25:01,246][_waitingSend:88] Sent: b'{"command": "login", "arguments": {"userId": "1959587", "password": "veGan/1102199c", "appName": ""}}'
[2022-05-12 16:25:01,676][_read:107] Received: {'status': True, 'streamSessionId': '262540fffee58c68-00004899-004bfdfc-d592f914c32785d3-833828dd'}
[2022-05-12 16:25:01,677][executeOrder:44] {'status': True, 'streamSessionId': '262540fffee58c68-00004899-004bfdfc-d592f914c32785d3-833828dd'}
[2022-05-12 16:25:01,678][_waitingSend:88] Sent: b'{"command": "getTickPrices", "arguments": {"level": 0, "symbols": ["EURUSD"], "timestamp": 165234750100}}'
[2022-05-12 16:25:01,883][_read:107] Received: {'status': True, 'returnData': {'quotations': [{'symbol': 'EURUSD', 'ask': 1.0436, 'bid': 1.04345, 'high': 1.05287, 'low': 1.0422, 'askVolume': 399000, 'bidVolume': 397000, 'timestamp': 1652347501372, 'level': 0, 'exemode': 1, 'spreadRaw': 0.00015, 'spreadTable': 1.5}]}}
[2022-05-12 16:25:01,887

sell price: 1.0438,                   ask: 1.0436,                   bid: 1.04345


[2022-05-12 16:25:02,109][_read:107] Received: {'status': True, 'returnData': {'order': 624500385}}
[2022-05-12 16:25:02,109][close:115] Closing socket
